# 为Claude构建自定义技能

学习如何创建、部署和管理自定义技能，以您组织的专业知识和工作流程扩展Claude的能力。

## 目录

1. [简介与设置](#introduction)
2. [理解自定义技能架构](#architecture)
3. [示例1：金融比率计算器](#financial-ratio)
4. [示例2：公司品牌指南](#brand-guidelines)
5. [示例3：金融建模套件](#financial-modeling)
6. [技能管理与版本控制](#management)
7. [最佳实践与生产提示](#best-practices)
8. [故障排除](#troubleshooting)

## 1. 简介与设置 {#introduction}

### 什么是自定义技能？

**自定义技能**是您创建的专业知识包，用于教授Claude您组织的独特工作流程、领域知识和最佳实践。与Anthropic的预构建技能（Excel、PowerPoint、PDF）不同，自定义技能允许您：

- **编纂组织知识** - 捕获您团队的具体方法
- **确保一致性** - 在所有交互中应用相同标准
- **自动化复杂工作流** - 将多步骤流程链接在一起
- **维护知识产权** - 保护专有方法的安全性

### 主要优势

| 优势 | 描述 |
|---------|-------------|
| **专业知识规模化** | 将专业知识部署到每个Claude交互中 |
| **版本控制** | 跟踪更改并在需要时回滚 |
| **可组合性** | 组合多个技能以完成复杂任务 |
| **隐私性** | 您的技能保持对组织私有 |

### 先决条件

在开始之前，请确保您已完成：
- [Notebook 1：技能入门](01_skills_introduction.ipynb)
- 具有技能Beta访问权限的Anthropic API密钥
- 安装了本地SDK的Python环境

### 环境设置

让我们设置环境并导入必要的库：

In [ ]:
import os
import sys
from pathlib import Path
from typing import Any

# 添加父目录用于导入
sys.path.insert(0, str(Path.cwd().parent))

from anthropic import Anthropic
from anthropic.lib import files_from_dir
from dotenv import load_dotenv

# 导入我们的工具
from file_utils import (
    download_all_files,
    extract_file_ids,
    print_download_summary,
)

# 我们稍后将在此notebook中创建skill_utils
# from skill_utils import (
#     create_skill,
#     list_skills,
#     delete_skill,
#     test_skill
# )

# 加载环境变量
load_dotenv(Path.cwd().parent / ".env")

API_KEY = os.getenv("ANTHROPIC_API_KEY")
MODEL = os.getenv("ANTHROPIC_MODEL", "claude-sonnet-4-5")

if not API_KEY:
    raise ValueError(
        "未找到ANTHROPIC_API_KEY。请将../.env.example复制到../.env并添加您的API密钥。"
    )

# 使用Skills beta初始化客户端
client = Anthropic(api_key=API_KEY, default_headers={"anthropic-beta": "skills-2025-10-02"})

# 设置目录
SKILLS_DIR = Path.cwd().parent / "custom_skills"
OUTPUT_DIR = Path.cwd().parent / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

print("✓ API密钥已加载")
print(f"✓ 使用模型：{MODEL}")
print(f"✓ 自定义技能目录：{SKILLS_DIR}")
print(f"✓ 输出目录：{OUTPUT_DIR}")
print("\n📝 为技能管理配置了Skills beta头部")

## 2. 理解自定义技能架构 {#architecture}

### 技能结构

每个自定义技能都遵循此目录结构：

```
skill_name/
├── SKILL.md           # 必需：带YAML前导的指令
├── *.md               # 可选：任何额外的.md文件（文档、指南）
├── scripts/           # 可选：可执行代码
│   ├── process.py
│   └── utils.js
└── resources/         # 可选：模板、数据文件
    └── template.xlsx
```

**重要：** 
- **SKILL.md是唯一必需的文件** - 其他所有内容都是可选的
- **允许多个.md文件** - 您可以在顶级文件夹中拥有任意数量的markdown文件
- **所有.md文件都会被加载** - 不仅加载SKILL.md和REFERENCE.md，还加载您包含的任何.md文件
- **按需组织** - 使用多个.md文件来构建复杂的文档结构

📖 阅读我们的工程博客文章[用Skills装备真实世界的智能体](https://www.anthropic.com/engineering/equipping-agents-for-the-real-world-with-agent-skills)

### 技能不仅仅是Markdown

![技能可以包括脚本和文件](../assets/not-just-markdown.png)

技能可以捆绑各种文件类型：
- **Markdown文件**：指令和文档（SKILL.md、REFERENCE.md等）
- **脚本**：Python、JavaScript或其他用于复杂操作的可执行代码
- **模板**：可自定义的预构建文件（Excel模板、文档模板）
- **资源**：支持数据文件、配置或资产

### SKILL.md要求

`SKILL.md`文件必须包括：

1. **YAML前导**（名称：64字符，描述：1024字符）
   - `name`：小写字母数字和连字符（必需）
   - `description`：技能功能的简要描述（必需）

2. **指令**（markdown格式）
   - 为Claude提供清晰的指导
   - 使用示例
   - 任何约束或规则
   - 推荐：保持在5,000令牌以下

### 额外的文档文件

您可以包含多个markdown文件以便更好地组织：

```
skill_name/
├── SKILL.md           # 主要指令（必需）
├── REFERENCE.md       # API参考（可选）
├── EXAMPLES.md        # 使用示例（可选）
├── TROUBLESHOOTING.md # 常见问题（可选）
└── CHANGELOG.md       # 版本历史（可选）
```

根目录中的所有`.md`文件在技能加载时都将对Claude可用。

### 捆绑文件示例

![技能中的捆绑文件](../assets/skills-bundled-files.png)

此示例展示技能如何捆绑多个文件：
- **SKILL.md**：包含主要指令，包括颜色、版式和部分
- **slide-decks.md**：特定用例的额外文档
- **脚本和资源**：可在技能执行期间引用和使用

### 渐进式披露

技能分三个阶段加载以优化令牌使用：

| 阶段 | 内容 | 令牌成本 | 加载时间 |
|-------|---------|------------|-------------|
| **1. 元数据** | 名称和描述 | 名称：64字符，描述：1024字符 | 始终可见 |
| **2. 指令** | 所有.md文件 | 推荐<5,000令牌 | 相关时 |
| **3. 资源** | 脚本和文件 | 按需 | 执行期间 |

### API工作流

```python
# 1. 创建技能
skill = client.beta.skills.create(
    display_title="My Skill",
    files=files_from_dir("path/to/skill")
)

# 2. 在消息中使用
response = client.beta.messages.create(
    container={
        "skills": [{
            "type": "custom",
            "skill_id": skill.id,
            "version": "latest"
        }]
    },
    # ... 消息参数的其他部分
)
```

### 最佳实践

有关技能创建和最佳实践的详细指导，请参阅：
- [Claude技能最佳实践](https://docs.claude.com/en/docs/agents-and-tools/agent-skills/best-practices)
- [技能文档](https://docs.claude.com/en/docs/agents-and-tools/agent-skills/overview)

### 创建技能工具函数

让我们创建技能管理的辅助函数：

In [ ]:
def create_skill(client: Anthropic, skill_path: str, display_title: str) -> dict[str, Any]:
    """
    从目录创建新的自定义技能。

    参数:
        client: Anthropic客户端实例
        skill_path: 技能目录路径
        display_title: 人类可读的技能名称

    返回:
        包含skill_id、版本和元数据的字典
    """
    try:
        # 使用files_from_dir创建技能
        skill = client.beta.skills.create(
            display_title=display_title, files=files_from_dir(skill_path)
        )

        return {
            "success": True,
            "skill_id": skill.id,
            "display_title": skill.display_title,
            "latest_version": skill.latest_version,
            "created_at": skill.created_at,
            "source": skill.source,
        }
    except Exception as e:
        return {"success": False, "error": str(e)}


def list_custom_skills(client: Anthropic) -> list[dict[str, Any]]:
    """
    列出工作区中的所有自定义技能。

    返回:
        技能字典列表
    """
    try:
        skills_response = client.beta.skills.list(source="custom")

        skills = []
        for skill in skills_response.data:
            skills.append(
                {
                    "skill_id": skill.id,
                    "display_title": skill.display_title,
                    "latest_version": skill.latest_version,
                    "created_at": skill.created_at,
                    "updated_at": skill.updated_at,
                }
            )

        return skills
    except Exception as e:
        print(f"列出技能时出错：{e}")
        return []


def delete_skill(client: Anthropic, skill_id: str) -> bool:
    """
    删除自定义技能及其所有版本。

    参数:
        client: Anthropic客户端
        skill_id: 要删除的技能ID

    返回:
        成功返回True，否则返回False
    """
    try:
        # 首先删除所有版本
        versions = client.beta.skills.versions.list(skill_id=skill_id)

        for version in versions.data:
            client.beta.skills.versions.delete(skill_id=skill_id, version=version.version)

        # 然后删除技能本身
        client.beta.skills.delete(skill_id)
        return True

    except Exception as e:
        print(f"删除技能时出错：{e}")
        return False


def test_skill(
    client: Anthropic,
    skill_id: str,
    test_prompt: str,
    model: str = "claude-sonnet-4-5",
) -> Any:
    """
    使用提示测试自定义技能。

    参数:
        client: Anthropic客户端
        skill_id: 要测试的技能ID
        test_prompt: 测试技能的提示
        model: 用于测试的模型

    返回:
        Claude的响应
    """
    response = client.beta.messages.create(
        model=model,
        max_tokens=4096,
        container={"skills": [{"type": "custom", "skill_id": skill_id, "version": "latest"}]},
        tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
        messages=[{"role": "user", "content": test_prompt}],
        betas=[
            "code-execution-2025-08-25",
            "files-api-2025-04-14",
            "skills-2025-10-02",
        ],
    )

    return response


print("✓ 技能工具函数已定义")
print("  - create_skill()")
print("  - list_custom_skills()")
print("  - delete_skill()")
print("  - test_skill()")

### 检查现有自定义技能

让我们看看您的工作区中是否已存在任何自定义技能：

### ⚠️ 重要：在开始之前清理现有技能

如果您正在重新运行此notebook，您可能拥有来自先前会话的技能。技能不能有重复的显示标题，因此您有三个选择：

1. **删除现有技能**（推荐用于测试） - 干净的方法
2. **使用不同的显示标题** - 在名称中添加时间戳或版本号
3. **使用新版本更新现有技能** - 参阅[技能管理与版本控制](#management)部分

让我们检查并可选地清理现有技能：

In [ ]:
# 检查可能冲突的现有技能
existing_skills = list_custom_skills(client)
skill_titles_to_create = [
    "Financial Ratio Analyzer",
    "Corporate Brand Guidelines",
    "Financial Modeling Suite",
]
conflicting_skills = []

if existing_skills:
    print(f"找到{len(existing_skills)}个现有自定义技能：")
    for skill in existing_skills:
        print(f"  - {skill['display_title']} (ID: {skill['skill_id']})")
        if skill["display_title"] in skill_titles_to_create:
            conflicting_skills.append(skill)

    if conflicting_skills:
        print(
            f"\n⚠️ 发现{len(conflicting_skills)}个与此notebook冲突的技能："
        )
        for skill in conflicting_skills:
            print(f"  - {skill['display_title']} (ID: {skill['skill_id']})")

        print("\n" + "=" * 70)
        print("要清理这些技能并重新开始，请取消注释并运行：")
        print("=" * 70)
        print("\n# 取消注释以下行以删除冲突的技能：")
        print("# for skill in conflicting_skills:")
        print("#     if delete_skill(client, skill['skill_id']):")
        print("#         print(f\"✅ 已删除：{skill['display_title']}\")")
        print("#     else:")
        print("#         print(f\"❌ 删除失败：{skill['display_title']}\")")

        # for skill in conflicting_skills:
        #     if delete_skill(client, skill['skill_id']):
        #         print(f"✅ 已删除：{skill['display_title']}")
        #     else:
        #         print(f"❌ 删除失败：{skill['display_title']}")
    else:
        print("\n✅ 未发现冲突的技能。可以继续！")
else:
    print("✅ 未发现现有自定义技能。可以创建新的！")

## 3. 示例1：金融比率计算器 {#financial-ratio}

让我们创建第一个自定义技能 - 一个可以分析公司财务健康的金融比率计算器。

### 技能概述

**金融比率计算器**技能将：
- 计算关键金融比率（ROE、P/E、流动比率等）
- 结合行业背景解释比率
- 生成格式化报告
- 与各种数据格式配合使用（CSV、JSON、文本）

### 上传金融分析器技能

现在让我们将金融分析器技能上传到Claude：

In [ ]:
# 上传金融分析器技能
financial_skill_path = SKILLS_DIR / "analyzing-financial-statements"

if financial_skill_path.exists():
    print("正在上传金融分析器技能...")
    result = create_skill(client, str(financial_skill_path), "Financial Ratio Analyzer")

    if result["success"]:
        financial_skill_id = result["skill_id"]
        print("✅ 技能上传成功！")
        print(f"   技能ID：{financial_skill_id}")
        print(f"   版本：{result['latest_version']}")
        print(f"   创建时间：{result['created_at']}")
    else:
        print(f"❌ 上传失败：{result['error']}")
        if "cannot reuse an existing display_title" in str(result["error"]):
            print("\n💡 解决方案：已存在同名技能。")
            print("   运行上面"清理现有技能"单元格先删除它，")
            print("   或将display_title更改为唯一名称。")
else:
    print(f"⚠️ 未找到技能目录：{financial_skill_path}")
    print(
        "请确保custom_skills目录包含analyzing-financial-statements文件夹。"
    )

### 测试金融分析器技能

让我们使用示例财务数据测试技能：

In [ ]:
# 测试金融分析器技能
if "financial_skill_id" in locals():
    test_prompt = """
    为这家公司计算财务比率：

    损益表：
    - 收入：$1,000M
    - EBITDA：$200M
    - 净利润：$120M

    资产负债表：
    - 总资产：$2,000M
    - 流动资产：$500M
    - 流动负债：$300M
    - 总债务：$400M
    - 股东权益：$1,200M

    市场数据：
    - 股价：$50
    - 流通股：100M

    请计算关键比率并提供分析。
    """

    print("正在测试金融分析器技能...")
    response = test_skill(client, financial_skill_id, test_prompt)

    # 打印响应
    for content in response.content:
        if content.type == "text":
            print(content.text)
else:
    print("⚠️ 请先上传金融分析器技能（运行前面的单元格）")

## 4. 示例2：公司品牌指南 {#brand-guidelines}

现在让我们创建一个确保所有文档遵循企业品牌标准的技能。

### 技能概述

**品牌指南**技能将：
- 应用一致的颜色、字体和布局
- 确保logo放置和使用
- 保持专业语调和信息传递
- 适用于所有文档类型（Excel、PowerPoint、PDF）

In [ ]:
# 上传品牌指南技能
brand_skill_path = SKILLS_DIR / "applying-brand-guidelines"

if brand_skill_path.exists():
    print("正在上传品牌指南技能...")
    result = create_skill(client, str(brand_skill_path), "Corporate Brand Guidelines")

    if result["success"]:
        brand_skill_id = result["skill_id"]
        print("✅ 技能上传成功！")
        print(f"   技能ID：{brand_skill_id}")
        print(f"   版本：{result['latest_version']}")
    else:
        print(f"❌ 上传失败：{result['error']}")
        if "cannot reuse an existing display_title" in str(result["error"]):
            print("\n💡 解决方案：已存在同名技能。")
            print("   运行上面"清理现有技能"单元格先删除它，")
            print("   或将display_title更改为唯一名称。")
else:
    print(f"⚠️ 未找到技能目录：{brand_skill_path}")

### 使用文档创建测试品牌指南

让我们通过创建品牌化的PowerPoint演示来测试品牌技能：

In [ ]:
# 测试带有PowerPoint创建的品牌指南技能
if "brand_skill_id" in locals():
    # 结合品牌技能和Anthropic的pptx技能
    response = client.beta.messages.create(
        model=MODEL,
        max_tokens=4096,
        container={
            "skills": [
                {"type": "custom", "skill_id": brand_skill_id, "version": "latest"},
                {"type": "anthropic", "skill_id": "pptx", "version": "latest"},
            ]
        },
        tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
        messages=[
            {
                "role": "user",
                "content": """创建一个包含3页的PowerPoint演示，遵循Acme公司品牌指南：

            第1页：标题页 - "2025年第四季度结果"
            第2页：收入概览 - 包含显示第一季度至第四季度增长的图表
            第3页：关键成就（3个要点）

            应用所有品牌颜色、字体和格式标准。
            """,
            }
        ],
        betas=[
            "code-execution-2025-08-25",
            "files-api-2025-04-14",
            "skills-2025-10-02",
        ],
    )

    print("Claude的响应：")
    for content in response.content:
        if content.type == "text":
            print(content.text[:500] + "..." if len(content.text) > 500 else content.text)

    # 下载生成的文件
    file_ids = extract_file_ids(response)
    if file_ids:
        results = download_all_files(
            client, response, output_dir=str(OUTPUT_DIR), prefix="branded_"
        )
        print_download_summary(results)
else:
    print("⚠️ 请先上传品牌指南技能")

## 5. 示例3：金融建模套件 {#financial-modeling}

让我们创建我们最高级的技能 - 用于估值和风险分析的综合金融建模套件。

### 技能概述

**金融建模套件**技能提供：
- **DCF估值**：完整的贴现现金流模型
- **敏感性分析**：测试变量对估值的影响
- **蒙特卡洛模拟**：带有概率分布的风险建模
- **情景规划**：最佳/基准/最差情况分析

这展示了具有复杂计算和专业级金融建模的多文件技能。

### 上传金融建模套件 

首先，上传金融建模技能：

In [ ]:
# 上传金融建模套件技能
modeling_skill_path = SKILLS_DIR / "creating-financial-models"

if modeling_skill_path.exists():
    print("正在上传金融建模套件技能...")
    result = create_skill(client, str(modeling_skill_path), "Financial Modeling Suite")

    if result["success"]:
        modeling_skill_id = result["skill_id"]
        print("✅ 技能上传成功！")
        print(f"   技能ID：{modeling_skill_id}")
        print(f"   版本：{result['latest_version']}")
        print("\n此技能包括：")
        print("   - DCF估值模型（dcf_model.py）")
        print("   - 敏感性分析框架（sensitivity_analysis.py）")
        print("   - 蒙特卡洛模拟功能")
        print("   - 情景规划工具")
    else:
        print(f"❌ 上传失败：{result['error']}")
else:
    print(f"⚠️ 未找到技能目录：{modeling_skill_path}")
    print(
        "请确保custom_skills目录包含creating-financial-models文件夹。"
    )

### 测试金融建模套件

让我们通过DCF估值请求测试高级建模功能：

In [ ]:
# 使用DCF估值测试金融建模套件
if "modeling_skill_id" in locals():
    dcf_test_prompt = """
    对TechCorp执行DCF估值，数据如下：

    历史财务（过去3年）：
    - 收入：$500M、$600M、$750M
    - EBITDA利润率：25%、27%、30%
    - 资本支出：$50M、$55M、$60M
    - 营运资本：收入的15%

    预测：
    - 收入增长：第1-3年20%，然后下降到第5年5%
    - EBITDA利润率扩张到第5年35%
    - 永续增长率：3%

    市场假设：
    - 加权平均资本成本：10%
    - 税率：25%
    - 当前净债务：$200M
    - 流通股：100M

    请创建完整的DCF模型，包含对加权平均资本成本和永续增长率的敏感性分析。
    生成包含以下内容的Excel文件：
    1. 收入预测
    2. 自由现金流计算
    3. 终值
    4. 企业价值到股权价值桥接
    5. 敏感性表
    """

    print("正在测试带有DCF估值的金融建模套件...")
    print("=" * 70)
    print("\n⏱️ 注意：复杂金融模型生成可能需要1-2分钟。\n")

    response = client.beta.messages.create(
        model=MODEL,
        max_tokens=4096,
        container={
            "skills": [
                {"type": "custom", "skill_id": modeling_skill_id, "version": "latest"},
                {"type": "anthropic", "skill_id": "xlsx", "version": "latest"},
            ]
        },
        tools=[{"type": "code_execution_20250825", "name": "code_execution"}],
        messages=[{"role": "user", "content": dcf_test_prompt}],
        betas=[
            "code-execution-2025-08-25",
            "files-api-2025-04-14",
            "skills-2025-10-02",
        ],
    )

    # 打印Claude的响应
    for content in response.content:
        if content.type == "text":
            # 打印前800个字符以保持输出可管理
            text = content.text
            if len(text) > 800:
                print(text[:800] + "\n\n[...为简洁起见输出被截断...]")
            else:
                print(text)

    # 如果生成，下载DCF模型
    file_ids = extract_file_ids(response)
    if file_ids:
        print("\n" + "=" * 70)
        print("正在下载生成的DCF模型...")
        results = download_all_files(
            client, response, output_dir=str(OUTPUT_DIR), prefix="dcf_model_"
        )
        print_download_summary(results)
        print("\n💡 打开Excel文件探索完整的DCF估值模型！")
else:
    print("⚠️ 请先上传金融建模套件技能（运行前面的单元格）")

## 8. 故障排除 {#troubleshooting}

In [ ]:
# 列出所有您的自定义技能
my_skills = list_custom_skills(client)

if my_skills:
    print(f"您有{len(my_skills)}个自定义技能：\n")
    print("=" * 70)
    for i, skill in enumerate(my_skills, 1):
        print(f"\n{i}. {skill['display_title']}")
        print(f"   技能ID：{skill['skill_id']}")
        print(f"   当前版本：{skill['latest_version']}")
        print(f"   创建时间：{skill['created_at']}")
        if skill.get("updated_at"):
            print(f"   最后更新：{skill['updated_at']}")
    print("\n" + "=" * 70)
else:
    print("您的工作区中未找到自定义技能。")

### 创建新版本

技能支持版本控制以维护历史并启用回滚。让我们为金融分析器技能进行增强并创建新版本。

#### 步骤1：增强金融分析器

我们将添加**医疗保健行业**基准，使我们的技能更加多样化。这是一个现实场景，您将根据用户需求扩展技能的能力。

In [ ]:
# 为金融分析器添加医疗保健行业基准
# 这演示了现实世界中的技能增强场景

if "financial_skill_id" in locals():
    # 读取当前的interpret_ratios.py文件
    interpret_file_path = SKILLS_DIR / "analyzing-financial-statements" / "interpret_ratios.py"

    with open(interpret_file_path) as f:
        content = f.read()

    # 在'manufacturing'部分之后添加医疗保健基准
    healthcare_benchmarks = """        },
        'healthcare': {
            'current_ratio': {'excellent': 2.3, 'good': 1.8, 'acceptable': 1.4, 'poor': 1.0},
            'debt_to_equity': {'excellent': 0.3, 'good': 0.6, 'acceptable': 1.0, 'poor': 1.8},
            'roe': {'excellent': 0.22, 'good': 0.16, 'acceptable': 0.11, 'poor': 0.07},
            'gross_margin': {'excellent': 0.65, 'good': 0.45, 'acceptable': 0.30, 'poor': 0.20},
            'pe_ratio': {'undervalued': 18, 'fair': 28, 'growth': 40, 'expensive': 55}
        """

    # 找到制造业部分之后和结束花括号之前的位置
    insert_pos = content.find("        }\n    }")  # 找到BENCHMARKS字典的结尾

    if insert_pos != -1:
        # 插入医疗保健基准
        new_content = content[:insert_pos] + healthcare_benchmarks + content[insert_pos:]

        # 保存增强的文件
        with open(interpret_file_path, "w") as f:
            f.write(new_content)

        print("✅ 已用医疗保健行业基准增强金融分析器")
        print("\n所做的更改：")
        print("  - 将医疗保健行业添加到BENCHMARKS")
        print("  - 包括特定阈值：")
        print("    • 流动比率（流动性）")
        print("    • 债务股本比（杠杆）")
        print("    • 股本回报率（盈利能力）")
        print("    • 毛利率")
        print("    • 市盈率（估值）")
        print("\n📝 现在我们可以使用此增强功能创建技能的新版本！")
    else:
        print("⚠️ 无法找到插入医疗保健基准的正确位置")
        print("文件结构可能已更改。")
else:
    print("⚠️ 请先上传金融分析器技能（运行第3节的单元格）")

#### 步骤2：创建新版本

现在我们已经增强了我们的技能，让我们创建一个新版本来跟踪此更改：

In [ ]:
# 为增强的金融分析器技能创建新版本
def create_skill_version(client: Anthropic, skill_id: str, skill_path: str):
    """为现有技能创建新版本。"""
    try:
        version = client.beta.skills.versions.create(
            skill_id=skill_id, files=files_from_dir(skill_path)
        )
        return {
            "success": True,
            "version": version.version,
            "created_at": version.created_at,
        }
    except Exception as e:
        return {"success": False, "error": str(e)}


# 使用我们的医疗保健增强创建新版本
if "financial_skill_id" in locals():
    print("正在创建带有医疗保健基准的金融分析器新版本...")

    result = create_skill_version(
        client, financial_skill_id, str(SKILLS_DIR / "analyzing-financial-statements")
    )

    if result["success"]:
        print("✅ 新版本创建成功！")
        print(f"   版本：{result['version']}")
        print(f"   创建时间：{result['created_at']}")
        print("\n📊 版本历史：")
        print("   v1：包含技术、零售、金融、制造业的原始技能")
        print(f"   v{result['version']}：使用医疗保健行业基准增强")
    else:
        print(f"❌ 版本创建失败：{result['error']}")
else:
    print("⚠️ 请运行前面的单元格上传技能并进行增强")

#### 步骤3：测试新版本

让我们通过分析医疗保健公司来验证我们的增强功能：

In [ ]:
# 使用医疗保健行业数据测试增强的技能
if "financial_skill_id" in locals():
    healthcare_test_prompt = """
    使用医疗保健行业基准分析这家医疗保健公司：

    公司：MedTech Solutions（医疗保健行业）

    损益表：
    - 收入：$800M
    - EBITDA：$320M
    - 净利润：$160M

    资产负债表：
    - 总资产：$1,200M
    - 流动资产：$400M
    - 流动负债：$200M
    - 总债务：$300M
    - 股东权益：$700M

    市场数据：
    - 股价：$75
    - 流通股：50M

    请计算关键比率并提供医疗保健特定分析。
    """

    print("正在使用医疗保健公司测试增强的金融分析器...")
    print("=" * 70)

    response = test_skill(client, financial_skill_id, healthcare_test_prompt, MODEL)

    # 打印Claude的分析
    for content in response.content:
        if content.type == "text":
            # 打印前1000个字符以保持输出可管理
            text = content.text
            if len(text) > 1000:
                print(text[:1000] + "\n\n[...为简洁起见输出被截断...]")
            else:
                print(text)

    print(
        "\n✅ 技能现在将'healthcare'识别为行业并应用特定基准！"
    )
else:
    print("⚠️ 请运行前面的单元格来创建增强版本")

### 清理：管理您的技能

当您完成测试或需要清理工作区时，您可以有选择地删除技能。让我们回顾我们创建的内容并提供清理选项：

In [ ]:
# 全面的技能清理与详细报告
def review_and_cleanup_skills(client, dry_run=True):
    """
    审查所有技能并可选地清理在此notebook中创建的技能。

    参数:
        client: Anthropic客户端
        dry_run: 如果为True，仅显示将被删除的内容而不实际删除
    """
    # 获取所有当前技能
    all_skills = list_custom_skills(client)

    # 我们在此notebook中创建的技能
    notebook_skill_names = [
        "Financial Ratio Analyzer",
        "Corporate Brand Guidelines",
        "Financial Modeling Suite",
    ]

    # 跟踪此notebook创建的技能
    notebook_skills = []
    other_skills = []

    for skill in all_skills:
        if skill["display_title"] in notebook_skill_names:
            notebook_skills.append(skill)
        else:
            other_skills.append(skill)

    print("=" * 70)
    print("技能清单报告")
    print("=" * 70)

    print(f"\n工作区中的自定义技能总数：{len(all_skills)}")

    if notebook_skills:
        print(f"\n📚 此notebook创建的技能（{len(notebook_skills)}）：")
        for skill in notebook_skills:
            print(f"   • {skill['display_title']}")
            print(f"     ID: {skill['skill_id']}")
            print(f"     版本: {skill['latest_version']}")
            print(f"     创建时间: {skill['created_at']}")
    else:
        print("\n✅ 未找到此notebook创建的技能")

    if other_skills:
        print(f"\n🔧 工作区中的其他技能（{len(other_skills)}）：")
        for skill in other_skills:
            print(f"   • {skill['display_title']} (v{skill['latest_version']})")

    # 清理选项
    if notebook_skills:
        print("\n" + "=" * 70)
        print("清理选项")
        print("=" * 70)

        if dry_run:
            print("\n🔍 试运行模式 - 不会删除任何技能")
            print("\n要删除notebook技能，取消注释并运行：")
            print("-" * 40)
            print("# review_and_cleanup_skills(client, dry_run=False)")
            print("-" * 40)

            print("\n这将删除：")
            for skill in notebook_skills:
                print(f"   • {skill['display_title']}")
        else:
            print("\n⚠️ 删除模式 - 技能将被永久删除")
            print("\n正在删除notebook技能...")

            success_count = 0
            for skill in notebook_skills:
                if delete_skill(client, skill["skill_id"]):
                    print(f"   ✅ 已删除：{skill['display_title']}")
                    success_count += 1
                else:
                    print(f"   ❌ 删除失败：{skill['display_title']}")

            print(f"\n📊 清理完成：{success_count}/{len(notebook_skills)}个技能已删除")

    return {
        "total_skills": len(all_skills),
        "notebook_skills": len(notebook_skills),
        "other_skills": len(other_skills),
        "notebook_skill_ids": [s["skill_id"] for s in notebook_skills],
    }


# 运行审查（默认为试运行模式）
print("正在审查您的自定义技能工作区...")
cleanup_summary = review_and_cleanup_skills(client, dry_run=True)

# 存储技能ID以备潜在的清理
if cleanup_summary["notebook_skill_ids"]:
    skills_to_cleanup = cleanup_summary["notebook_skill_ids"]
    print(f"\n💡 提示：测试完成后可以清理{len(skills_to_cleanup)}个技能")

# 取消注释下面的行以实际删除notebook技能：
# review_and_cleanup_skills(client, dry_run=False)

## 7. 最佳实践与生产提示 {#best-practices}

### 技能设计原则

1. **单一责任**：每个技能应专注于一个专业领域
2. **清晰的文档**：SKILL.md应该全面但简洁
3. **错误处理**：脚本应该优雅地处理边缘情况
4. **版本控制**：使用Git跟踪技能更改
5. **测试**：在生产部署前始终测试技能

### 目录结构最佳实践

```
custom_skills/
├── financial_analyzer/       # 单一目的，清晰的命名
│   ├── SKILL.md             # 少于5,000令牌
│   ├── scripts/             # 模块化Python/JS文件
│   └── tests/               # 脚本的单元测试
├── brand_guidelines/         # 组织标准
│   ├── SKILL.md
│   ├── REFERENCE.md         # 额外文档
│   └── assets/              # 徽标、模板
```

### 性能优化

| 策略 | 影响 | 实施 |
|----------|--------|----------------|
| **最小化前导** | 更快的技能发现 | 名称：64字符，描述：1024字符 |
| **延迟加载** | 减少令牌使用 | 仅在需要时引用文件 |
| **技能组合** | 避免重复 | 组合技能 vs 超级技能 |
| **缓存** | 更快响应 | 重用技能容器 |

### 安全考虑

- **API密钥**：绝不在技能中硬编码凭据
- **数据隐私**：不要在技能文件中包含敏感数据
- **访问控制**：技能特定于工作区
- **验证**：清理脚本中的输入
- **审计跟踪**：记录技能使用情况以确保合规性

## 下一步

🎉 **恭喜！** 您已经学会了如何为Claude创建、部署和管理自定义技能。

### 您学到的内容

- ✅ 自定义技能架构和要求
- ✅ 使用SKILL.md和Python脚本创建技能
- ✅ 通过API上传技能
- ✅ 组合自定义和Anthropic技能
- ✅ 生产部署的最佳实践
- ✅ 故障排除常见问题

### 继续您的旅程

1. **实验**：修改示例技能以适应您的用例
2. **构建**：为您的组织工作流程创建技能
3. **优化**：监控令牌使用情况和性能
4. **共享**：为团队协作记录您的技能

### 资源

- [Claude API文档](https://docs.anthropic.com/en/api/messages)
- [技能文档](https://docs.claude.com/en/docs/agents-and-tools/agent-skills/overview)
- [最佳实践](https://docs.claude.com/en/docs/agents-and-tools/agent-skills/best-practices)
- [文件API文档](https://docs.claude.com/en/api/files-content)
- 示例技能仓库（即将推出）

### 值得尝试的技能创意

- 📊 **数据管道**：带有验证的ETL工作流
- 📝 **文档模板**：合同、提案、报告
- 🔍 **代码审查**：样式指南和最佳实践
- 📈 **分析仪表板**：KPI跟踪和可视化
- 🤖 **自动化套件**：重复任务工作流

快乐构建技能！🚀